In [147]:

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Normalization
import os
import pandas as pd
import numpy as np
from predict import preprocess, train_test_split, df_yielder, date_to_int
from sklearn.preprocessing import MinMaxScaler, StandardScaler

TRAIN_RATIO = 0.8
TIMESTEP = 12


In [148]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    
    return model

In [165]:
df = pd.read_csv('data/yf_attributes/AAPL.csv')
df['Date'] = [date_to_int(date) for date in df['Date']]

X_sc = MinMaxScaler()
scaled_X = X_sc.fit_transform(df)

targets = df[['Close']]
y_sc = MinMaxScaler()
scaled_y = y_sc.fit_transform(targets)

unscaled_X = df.values.tolist()
unscaled_y = targets['Close']

In [150]:
minimum_y, maximum_y = min(targets['Close']), max(targets['Close'])
print(maximum_y)
def descale_prediction(predicted_value):
    return predicted_value * (maximum_y - minimum_y) + minimum_y

196.4499969482422


In [151]:
def create_sequences(scaled_X, scaled_y, timestep):
    X, y = [], []
    for i in range(len(scaled_X) - timestep):
        X.append(scaled_X[i:i + timestep]) #Using all data between timesteps
        y.append(scaled_y[i + timestep]) #Using next timesteps close data as target
    
    return np.array(X), np.array(y)

In [169]:

X, y = create_sequences(scaled_X, scaled_y, TIMESTEP)

split_point = int(TRAIN_RATIO * len(X))
X_train, y_train = X[:split_point], y[:split_point]
X_test, y_test = X[split_point:], y[split_point:]


In [170]:
model = create_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
12/12 [==============================] - 2s 7ms/step - loss: 0.0011
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 3.6113e-04
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 2.0140e-04
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: 1.1050e-04
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 6.8867e-05
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 5.5634e-05
Epoch 7/100
12/12 [==============================] - 0s 6ms/step - loss: 4.9890e-05
Epoch 8/100
12/12 [==============================] - 0s 7ms/step - loss: 4.8602e-05
Epoch 9/100
12/12 [==============================] - 0s 7ms/step - loss: 5.0267e-05
Epoch 10/100
12/12 [==============================] - 0s 6ms/step - loss: 5.3903e-05
Epoch 11/100
12/12 [==============================] - 0s 7ms/step - loss: 5.6971e-05
Epoch 12/100
12/12 [==============================] - 0s 7ms/step - loss: 5.97

In [171]:

loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

3/3 [==============================] - 0s 4ms/step - loss: 0.0832
Test Loss: 0.08320264518260956


In [177]:

last_sequence = X_test[-1].reshape(1, X_test[-1].shape[0], X_test[-1].shape[1])

scaled_prediction = model.predict(last_sequence)
x = y_sc.inverse_transform(scaled_prediction)


print(x)




1/1 [==============================] - 0s 25ms/step
[[78.909294]]


In [22]:
#TODO check stackoverfflow to scale issues

(12, 7)